# Chapter 3.4 Onwards

## 3.38
1.

$(peter,paul,mary)=\frac{100+10-20}{2}=45$

$(peter,mary,paul)=\frac{100+10}{2}-20=35$

$(paul,peter,mary)=\frac{100-20+10}{2}=45$

$(paul,mary,peter)=\frac{100-20}{2}+10=50$

$(mary,peter,paul)=\frac{100}{2}+10-20=40$

$(mary,paul,peter)=\frac{100}{2}-20+10=40$

2.

```
|    Peter    |     Paul    | Bank |    Mary    |
|:-----------:|:-----------:|:----:|:----------:|
|             |             |  100 |            |
| Access: 100 | Access: 100 |      | Access:100 |
|   New: 100  |   New: 80   |      |   New: 50  |
|   Set: 110  |             |      |            |
|             |             | 110  |            |
|             |             |      | Set: 50    |
|             |             | 50   |            |
|             | Set: 80     |      |            |
|             |             | 80   |            |


|    Peter    |     Paul    | Bank |    Mary    |
|:-----------:|:-----------:|:----:|:----------:|
|             |             |  100 |            |
| Access: 100 | Access: 100 |      | Access:100 |
|   New: 110  |   New: 80   |      |   New: 50  |
|             |             |      |   Set: 50  |
|             |             | 50   |            |
|             |   Set: 80   |      |            |
|             |             | 80   |            |
|  Set: 110   |             |      |            |
|             |             | 110  |            |
```

## 3.39

In [ ]:
; n.b make-serializer not implemented yet
(define x 10)
(define s (make-serializer))
(parallel-execute
 (lambda ()
   (set! x ((s (lambda () (* x x)))))) ; P1
 (s (lambda () (set! x (+ x 1))))) ; P2

Possible values:
* `101`: 
    * $P_1$ sets `x` to `100`
    * $P_2$ increments `x` to `101`
* `121`:
    * $P_2$ increments `x` to `11`
    * $P_1$ sets `x` to `x` times `x`
* `100`:
    * $P_1$ accesses `x` (twice)
    * $P_2$ sets `x` to `11`
    * $P_1$ sets `x`

$P_2$ can interleave between the *computation* and *assignment* in $P_1$.

## 3.40

In [ ]:
(define x 10)
(parallel-execute
 (lambda () (set! x (* x x)))
 (lambda () (set! x (* x x x))))

Possible values:
* `100` (Steps 1 and 2 can be swapped):
    1. $P_1$ accesses `x` (`2` times)
    2. $P_2$ accesses `x` (`3` times)
    3. $P_2$ sets `x` to `1000`
    4. $P_1$ sets `x` to `100`
* `1000` (Steps 1 and 2 can be swapped):
    1. $P_1$ accesses `x` (`2` times)
    2. $P_2$ accesses `x` (`3` times)
    3. $P_1$ sets `x` to `100`
    4. $P_2$ sets `x` to `1000`
* `10000`: 
    *  
        1. $P_1$ accesses `x` (`1` time)
        2. $P_2$ accesses `x` (`3` times)
        3. $P_2$ sets `x` to `1000`
        4. $P_1$ accesses `x` (`1` time) -> `1000`
        5. $P_1$ sets `x` to `10000`
    * 
        1. $P_2$ accesses `x` (`2` times)
        2. $P_1$ accesses `x` (`2` times)
        3. $P_1$ sets `x` to `100`
        4. $P_2$ accesses `x` (`1` time) -> `100`
        5. $P_2$ sets `x` to `10000` -> `(* 10 10 100)`
* `100000`:
    1. $P_2$ accesses `x` (`1` time)
    2. $P_1$ accesses `x` (`2` times)
    3. $P_1$ sets `x` to `100`
    4. $P_2$ accesses `x` (`2` times) -> `100`
    5. $P_2$ sets `x` to `100000` -> `(* 10 100 100)`
* `1000000`:
    * 
        1. $P_2$ accesses `x` (`3` times)
        2. $P_2$ sets `x` to `1000`
        3. $P_1$ accesses `x` (`2` times)
        4. $P_1$ sets `x` to `1000000` -> `(* 1000 1000)`
    * 
        1. $P_1$ accesses `x` (`2` times)
        2. $P_1$ sets `x` to `100`
        3. $P_2$ accesses `x` (`3` times) -> `100`
        4. $P_2$ sets `x` to `1000000` -> `(* 100 100 100)`
       

In [ ]:
(define x 10)
(define s (make-serializer))
(parallel-execute
 (s (lambda () (set! x (* x x))))
 (s (lambda () (set! x (* x x x)))))

`1000000` is the only possible value as $P_1$ and $P_2$ cannot interleave due to the serializer. Execution must follow one of the two paths shown for the `1000000` value in part 1.

## 3.41
No, unserialized access to the balance is acceptable because, in the given implementation, accessing the balance does not alter it's value and is completed in **a single step**.

If accessing the balance required multiple steps, then an unserialized implementation could result in anomalous behaviour as other processes could be interleaved between the steps. 

## 3.42

In [ ]:
; creates new serialized procedure on each withdraw/deposit message
(define (make-account balance)
  (define (withdraw amount)
    (if (>= balance amount)
        (begin (set! balance
                     (- balance amount))
               balance)
        "Insufficient funds"))
  (define (deposit amount)
    (set! balance (+ balance amount))
    balance)
  (let ((protected (make-serializer)))
    (define (dispatch m)
      (cond ((eq? m 'withdraw)
             (protected withdraw))
            ((eq? m 'deposit)
             (protected deposit))
            ((eq? m 'balance)
             balance)
            (else (error "make-account.dispatch"
                         "Unknown request: ~a"
                         m))))
    dispatch))

; returns same serialized procedure that was created at same time as account
(define (make-account balance)
  (define (withdraw amount)
    (if (>= balance amount)
        (begin (set! balance
                     (- balance amount))
               balance)
        "Insufficient funds"))
  (define (deposit amount)
    (set! balance (+ balance amount))
    balance)
  (let ((protected (make-serializer)))
    (let ((protected-withdraw
           (protected withdraw))
          (protected-deposit
           (protected deposit)))
      (define (dispatch m)
        (cond ((eq? m 'withdraw)
               protected-withdraw)
              ((eq? m 'deposit)
               protected-deposit)
              ((eq? m 'balance)
               balance)
              (else (error "make-account.dispatch"
                           "Unknown request: ~a"
                           m))))
        dispatch)))

Yes, the change is safe - there is no difference in the concurrency that is allowed because the procedures are still handled by the **same serializer**. 